# CLRTAP review 2023

Main script for the CLRTAP review in 2023. 

In [1]:
%%capture
from arcgis.gis import GIS
from IPython.display import display
mygis = GIS("pro") # Use ArcGIS Pro to connect
myaccount = mygis.users.me
myaccount

import numpy as np
import os
import csv
import arcpy
import pandas as pd
from arcpy import env
from arcpy.sa import *
from arcgis.features import GeoAccessor, GeoSeriesAccessor


# Load sqlalchemy
import sqlalchemy as db
from sqlalchemy import create_engine

# Define gdb
path_to_gdb = "Q:/Delivery/GISdata/CLRTAP_review/gdb/clrtap-review.gdb"
path_to_emissions_gdb = "Q:/Delivery/GISdata/CLRTAP_review/gdb/clrtap-review-emissions.gdb"

# Countries to be reviewed by Ricardo for this submission
countries = ['Austria',
             'Bulgaria',
             'Italy',
             'Poland',
             'Romania',
             'Slovakia',
             'Spain',
             'Sweden',
             'Switzerland',
             'Czech_Republic',
             'Finland',
             'Lithuania']     

## Convert both CLRTAP and IASA rasters to points

In [15]:


# CLRTAP grids

# Set the current workspace
clrtap_raster_path = r"Q:/Delivery/GISdata/CLRTAP_review/data/Raster"
arcpy.env.workspace = clrtap_raster_path
# Get and print a list of GRIDs from the workspace
rasters_clrtap = arcpy.ListRasters("*", "TIF")
for r in rasters_clrtap:
    input_raster = os.path.join(clrtap_raster_path, r)
    output_raster = os.path.join(path_to_gdb, r[:-4])
    print("Exporting " + r[:-4])
    with arcpy.EnvManager(extent='-30.0000000001998 29.9999999995004 89.9000000002998 81.9999999999001 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'):
        arcpy.conversion.RasterToPoint(
            in_raster = input_raster,
            out_point_features = output_raster,
            raster_field="Value"
        )    

# IASA grids

# Set the current workspace
iasa_raster_path = r"Q:/Delivery/GISdata/CLRTAP_review/data/Raster/IASA_agri_K_L"
arcpy.env.workspace = iasa_raster_path
# Get and print a list of GRIDs from the workspace
rasters_iasa = arcpy.ListRasters("*", "TIF")
for r in rasters_iasa:
    input_raster = os.path.join(iasa_raster_path, r)
    output_raster = os.path.join(path_to_gdb, r[:-4])
    print("Exporting " + r[:-4])
    with arcpy.EnvManager(extent='-30.0000000001998 29.9999999995004 89.9000000002998 81.9999999999001 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'):
        arcpy.conversion.RasterToPoint(
            in_raster = input_raster,
            out_point_features = output_raster,
            raster_field="Value"
        )     
    



Exporting emep_clrtap_NH3_K
Exporting emep_clrtap_NH3_L
Exporting emep_clrtap_NMVOC_K
Exporting emep_clrtap_NMVOC_L
Exporting emep_clrtap_NOx_K
Exporting emep_clrtap_NOx_L
Exporting emep_clrtap_PM2_5_K
Exporting emep_clrtap_PM2_5_L
Exporting Agr_emis_gnfr_NH3_K_2015
Exporting Agr_emis_gnfr_NH3_K_2020
Exporting Agr_emis_gnfr_NH3_L_2015
Exporting Agr_emis_gnfr_NH3_L_2020
Exporting Agr_emis_gnfr_NOx_K_2015
Exporting Agr_emis_gnfr_NOx_K_2020
Exporting Agr_emis_gnfr_NOx_L_2015
Exporting Agr_emis_gnfr_NOx_L_2020
Exporting Agr_emis_gnfr_PM10_K_2015
Exporting Agr_emis_gnfr_PM10_K_2020
Exporting Agr_emis_gnfr_PM10_L_2015
Exporting Agr_emis_gnfr_PM10_L_2020
Exporting Agr_emis_gnfr_PM25_K_2015
Exporting Agr_emis_gnfr_PM25_K_2020
Exporting Agr_emis_gnfr_PM25_L_2015
Exporting Agr_emis_gnfr_PM25_L_2020
Exporting Agr_emis_gnfr_VOC_K_2015
Exporting Agr_emis_gnfr_VOC_K_2020
Exporting Agr_emis_gnfr_VOC_L_2015
Exporting Agr_emis_gnfr_VOC_L_2020


In [4]:
# Set the workspace and input feature class
arcpy.env.workspace = path_to_gdb
in_fc = "World_borders"

# Get a list of unique values from a specific field
field = "CNTRY_NAME"
unique_values = sorted({row[0] for row in arcpy.da.SearchCursor(in_fc, [field])})

# Iterate through the unique values and export the corresponding feature class
for value in unique_values:
    print(value)
    out_fc = "border_{}".format(value)
    arcpy.Select_analysis(in_fc, out_fc, "{} = '{}'".format(arcpy.AddFieldDelimiters(in_fc, field), value))

Austria
Bulgaria
Czech_Republic
Finland
Italy
Lithuania
Poland
Romania
Slovakia
Spain
Sweden
Switzerland


In [ ]:
arcpy.analysis.SpatialJoin(
    target_features="EMEP_GRID",
    join_features=r"Borders\border_Austria",
    out_feature_class=r"Q:\Delivery\GISdata\CLRTAP_review\gdb\clrtap-review.gdb\emep_austria",
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping='OBJECTID "OBJECTID" true true false 10 Long 0 10,First,#,EMEP_GRID,OBJECTID,-1,-1;long_min "long_min" true true false 19 Double 0 0,First,#,EMEP_GRID,long_min,-1,-1;long_max "long_max" true true false 19 Double 0 0,First,#,EMEP_GRID,long_max,-1,-1;lat_min "lat_min" true true false 19 Double 0 0,First,#,EMEP_GRID,lat_min,-1,-1;lat_max "lat_max" true true false 19 Double 0 0,First,#,EMEP_GRID,lat_max,-1,-1;long "long" true true false 19 Double 0 0,First,#,EMEP_GRID,long,-1,-1;lat "lat" true true false 19 Double 0 0,First,#,EMEP_GRID,lat,-1,-1;ISO "ISO" true true false 3 Text 0 0,First,#,EMEP_GRID,ISO,0,3;fraction "fraction" true true false 19 Double 0 0,First,#,EMEP_GRID,fraction,-1,-1;area "area" true true false 19 Double 0 0,First,#,EMEP_GRID,area,-1,-1;frac_area "frac_area" true true false 19 Double 0 0,First,#,EMEP_GRID,frac_area,-1,-1;Name "Name" true true false 70 Text 0 0,First,#,EMEP_GRID,Name,0,70;Number "Number" true true false 5 Long 0 5,First,#,EMEP_GRID,Number,-1,-1',
    match_option="INTERSECT",
    search_radius=None,
    distance_field_name=""
)

In [10]:
# Export EMEP grids per country

for country in countries:
    join_name = os.path.join(path_to_gdb, "border_" + country)
    out_feature_class_name = os.path.join(path_to_gdb, "emep_" + country)
    print("Exporting " + "emep_" + country)
    arcpy.analysis.SpatialJoin(
        target_features="EMEP_GRID",
        join_features=join_name,
        out_feature_class=out_feature_class_name,
        join_operation="JOIN_ONE_TO_ONE",
        join_type="KEEP_COMMON",
        field_mapping='OBJECTID "OBJECTID" true true false 10 Long 0 10,First,#,EMEP_GRID,OBJECTID,-1,-1;long_min "long_min" true true false 19 Double 0 0,First,#,EMEP_GRID,long_min,-1,-1;long_max "long_max" true true false 19 Double 0 0,First,#,EMEP_GRID,long_max,-1,-1;lat_min "lat_min" true true false 19 Double 0 0,First,#,EMEP_GRID,lat_min,-1,-1;lat_max "lat_max" true true false 19 Double 0 0,First,#,EMEP_GRID,lat_max,-1,-1;long "long" true true false 19 Double 0 0,First,#,EMEP_GRID,long,-1,-1;lat "lat" true true false 19 Double 0 0,First,#,EMEP_GRID,lat,-1,-1;ISO "ISO" true true false 3 Text 0 0,First,#,EMEP_GRID,ISO,0,3;fraction "fraction" true true false 19 Double 0 0,First,#,EMEP_GRID,fraction,-1,-1;area "area" true true false 19 Double 0 0,First,#,EMEP_GRID,area,-1,-1;frac_area "frac_area" true true false 19 Double 0 0,First,#,EMEP_GRID,frac_area,-1,-1;Name "Name" true true false 70 Text 0 0,First,#,EMEP_GRID,Name,0,70;Number "Number" true true false 5 Long 0 5,First,#,EMEP_GRID,Number,-1,-1',
        match_option="INTERSECT",
        search_radius=None,
        distance_field_name=""
    )
print("Done")

Exporting emep_Austria
Exporting emep_Bulgaria
Exporting emep_Italy
Exporting emep_Poland
Exporting emep_Romania
Exporting emep_Slovakia
Exporting emep_Spain
Exporting emep_Sweden
Exporting emep_Switzerland
Exporting emep_Czech_Republic
Exporting emep_Finland
Exporting emep_Lithuania
Done


In [15]:
rasters =  ['emep_clrtap_NH3_K',        
            'emep_clrtap_NH3_L',     
            'emep_clrtap_NMVOC_K',   
            'emep_clrtap_NMVOC_L',    
            'emep_clrtap_NOx_K',       
            'emep_clrtap_NOx_L',       
            'emep_clrtap_PM2_5_K',    
            'emep_clrtap_PM2_5_L',     
            'Agr_emis_gnfr_NH3_K_2015', 
            'Agr_emis_gnfr_NH3_K_2020',     
            'Agr_emis_gnfr_NH3_L_2015',      
            'Agr_emis_gnfr_NH3_L_2020',       
            'Agr_emis_gnfr_NOx_K_2015',      
            'Agr_emis_gnfr_NOx_K_2020',      
            'Agr_emis_gnfr_NOx_L_2015',      
            'Agr_emis_gnfr_NOx_L_2020',      
            'Agr_emis_gnfr_PM10_K_2015',    
            'Agr_emis_gnfr_PM10_K_2020',   
            'Agr_emis_gnfr_PM10_L_2015',   
            'Agr_emis_gnfr_PM10_L_2020',    
            'Agr_emis_gnfr_PM25_K_2015',  
            'Agr_emis_gnfr_PM25_K_2020',   
            'Agr_emis_gnfr_PM25_L_2015',    
            'Agr_emis_gnfr_PM25_L_2020',   
            'Agr_emis_gnfr_VOC_K_2015',     
            'Agr_emis_gnfr_VOC_K_2020',     
            'Agr_emis_gnfr_VOC_L_2015',     
            'Agr_emis_gnfr_VOC_L_2020']

for c in countries:
    target_path = os.path.join(path_to_gdb, "emep_" + c)
    for r in rasters:
        join_feature_path = os.path.join(path_to_gdb, r)
        out_feature_path = os.path.join(path_to_emissions_gdb, c + "_" + r)
        print("Exporting " + c + "_" + r)
        arcpy.analysis.SpatialJoin(
        target_features = target_path,
        join_features = join_feature_path,
        out_feature_class = out_feature_path,
        join_operation="JOIN_ONE_TO_ONE",
        join_type="KEEP_ALL",
        match_option="INTERSECT",
        search_radius=None,
        distance_field_name=""
)
print("DONE !!!")

Exporting Austria_emep_clrtap_NH3_K
Exporting Austria_emep_clrtap_NH3_L
Exporting Austria_emep_clrtap_NMVOC_K
Exporting Austria_emep_clrtap_NMVOC_L
Exporting Austria_emep_clrtap_NOx_K
Exporting Austria_emep_clrtap_NOx_L
Exporting Austria_emep_clrtap_PM2_5_K
Exporting Austria_emep_clrtap_PM2_5_L
Exporting Austria_Agr_emis_gnfr_NH3_K_2015
Exporting Austria_Agr_emis_gnfr_NH3_K_2020
Exporting Austria_Agr_emis_gnfr_NH3_L_2015
Exporting Austria_Agr_emis_gnfr_NH3_L_2020
Exporting Austria_Agr_emis_gnfr_NOx_K_2015
Exporting Austria_Agr_emis_gnfr_NOx_K_2020
Exporting Austria_Agr_emis_gnfr_NOx_L_2015
Exporting Austria_Agr_emis_gnfr_NOx_L_2020
Exporting Austria_Agr_emis_gnfr_PM10_K_2015
Exporting Austria_Agr_emis_gnfr_PM10_K_2020
Exporting Austria_Agr_emis_gnfr_PM10_L_2015
Exporting Austria_Agr_emis_gnfr_PM10_L_2020
Exporting Austria_Agr_emis_gnfr_PM25_K_2015
Exporting Austria_Agr_emis_gnfr_PM25_K_2020
Exporting Austria_Agr_emis_gnfr_PM25_L_2015
Exporting Austria_Agr_emis_gnfr_PM25_L_2020
Exportin

## Calculate emissions using the bordering fraction

In [ ]:
arcpy.env.workspace = path_to_emissions_gdb
country_emissions = arcpy.ListFeatureClasses("*")
# country_emissions = ['Lithuania_Agr_emis_gnfr_VOC_L_2020', 'Finland_Agr_emis_gnfr_PM25_L_2020', 
#                      'Czech_Republic_Agr_emis_gnfr_PM25_L_2020' ]
sdf = []
# df = []
for ce in country_emissions:
#     print(ce)
    in_path = os.path.join(path_to_emissions_gdb, ce)
    # Create a feature layer object
    # fl = arcpy.MakeFeatureLayer_management(feature_layer, "fl_lyr")
    
    # Convert the feature layer to a spatially enabled dataframe
    sdf = pd.DataFrame.spatial.from_featureclass(in_path)
    sdf = sdf.drop(['SHAPE','Join_Count', 'TARGET_FID', 'Join_Count_1', 'TARGET_FID_1', 'TARGET_FID',
                    'OBJECTID_1', 'long_min','long_max','lat_min','lat_max','long', 'lat', 'Number', 'Name'], axis=1)
    sdf = sdf.fillna(0)
    sdf['emissions'] = sdf.apply(lambda row: row['fraction'] * row['grid_code'], axis=1)
    df = df.append(sdf, ignore_index=True)
    df = df.fillna(0)
print("")
# Print the first five rows of the dataframe
print(df)


[86]:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
[86]:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
[86]:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
[86]:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
[86]:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
[86]:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
[86]:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
[86]:1

[86]:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
[86]:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
[86]:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
[86]:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
[86]:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
[86]:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
[86]:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
[86]:1

In [ ]:
sums = df.groupby('source')['emissions'].sum()

# print the resulting Series
print(sums)

In [ ]:
sums.to_csv('my_dataframe.csv', index=False)